In [1]:
import os, sys, pickle
import numpy as np
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path = list(set(sys.path))
import matplotlib.pyplot as plt
%matplotlib inline

from results_path import DDPG_PATH, PPO_PATH

from common.evaluate import make_vis_env, test_env

from flow.multiagent_envs import MultiWaveAttenuationMergePOEnv
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from ray.tune import run_experiments

import gym, ray
from ray.rllib.agents.ppo import PPOAgent, DEFAULT_CONFIG
from ray.rllib.agents.ddpg import DDPGAgent, DEFAULT_CONFIG
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

benchmark_name = 'multi_merge'
exp_name = '750s_multi_merge'
AGENT = 'PPO' # 'PPO' or 'DDPG' or 'HUMAN'
checkpoint = '50'

In [2]:
print(PPO_PATH)
print(DDPG_PATH)

PPO_MultiWaveAttenuationMergePOEnv-v0_1_entropy_coeff=0.01,observation_filter=MeanStdFilter_2019-04-23_17-32-43oiiqbhe1
DDPG_MultiWaveAttenuationMergePOEnv-v0_0_actor_hiddens=[64, 64],critic_hiddens=[64, 64],lr=0.001,observation_filter=MeanStdFilter_2019-04-23_06-08-35_b17350n


In [3]:
PPO_PATH = 'PPO_MultiWaveAttenuationMergePOEnv-v0_[eta1, eta2]:[1.0, 3.0]_t_min:10.0_6_2019-04-26_11-08-29ix58dnlj'

In [4]:
# PPO_PATH = exp_name + '/' + PPO_PATH

In [5]:
ray.init(num_cpus=1, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-26_17-10-52_425/logs.
Waiting for redis server at 127.0.0.1:33629 to respond...
Waiting for redis server at 127.0.0.1:62428 to respond...
Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '10.138.0.2',
 'object_store_addresses': ['/tmp/ray/session_2019-04-26_17-10-52_425/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-04-26_17-10-52_425/sockets/raylet'],
 'redis_address': '10.138.0.2:33629',
 'webui_url': ''}

In [6]:
benchmark = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.flow_params
horizon = flow_params['env'].horizon

def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

create_env, env_name = make_create_env(params=flow_params, version=0)

In [7]:
# Register as rllib env
register_env(env_name, create_env)

In [8]:
if AGENT == 'HUMAN':
    PATH = ''
else:
    if AGENT == 'PPO':
        PATH = PPO_PATH
    if AGENT == 'DDPG':
        PATH = DDPG_PATH

    config_path = '/headless/rl_project/ray_results/750s_multi_merge/' + PATH + '/params.pkl'
    checkpoint_path = '/headless/rl_project/ray_results/750s_multi_merge/' + PATH + '/checkpoint_{}/checkpoint-{}'.format(checkpoint, checkpoint)

    with open(config_path, mode='rb') as f:
        config = pickle.load(f)

    if AGENT == 'PPO':
        agent = PPOAgent(config=config, env=env_name)
    elif AGENT == 'DDPG':
        agent = DDPGAgent(config=config, env=env_name)
        
    agent.restore(checkpoint_path)

Created LogSyncer for /headless/ray_results/PPO_MultiWaveAttenuationMergePOEnv-v0_2019-04-26_17-10-53_dhrgp3x -> None
 Starting SUMO on port 45381
2019-04-26 17:10:54,744	INFO policy_evaluator.py:262 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/opt/conda/envs/flow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
 Starting SUMO on port 45381
2019-04-26 17:10:59,690	INFO multi_gpu_optimizer.py:74 -- LocalMultiGPUOptimizer devices ['/cpu:0']
/opt/conda/envs/flow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
env = make_vis_env(flow_params, MultiWaveAttenuationMergePOEnv, MergeScenario)
# env = create_env()

 Starting SUMO on port 47579


Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 2 seconds


In [10]:
# calculate the space-time velocity map
left_length = env.k.scenario.edge_length('left')
car_length = 5.0
scale = 10
vel_lists = []
vel_map_lists = []
outflow_lists = []

for _ in range(6):
    state = env.reset()
    vel = []
    outflow = []
    for i in range(env.env_params.horizon):
        # record the mean velocity
        v = np.mean(env.k.vehicle.get_speed(env.k.vehicle.get_ids()))
        vel.append(v)

        # record the velocity map
        ids = env.k.vehicle.get_ids()
        vel_map_ = np.zeros(int(left_length)*scale)
        for id_ in ids:
            pos_ = np.round(env.k.vehicle.get_position(id_), decimals=1)
            vel_ = env.k.vehicle.get_speed(id_)
            pos_bottom = max(0, int((pos_-car_length/2.0)*scale))
            pos_top = min(int(left_length)*scale, int((pos_+car_length/2.0)*scale))
            vel_map_[pos_bottom:pos_top] = vel_            

        # step the simulation
        rl_ids = env.k.vehicle.get_rl_ids()
        actions = {}
        if AGENT != 'HUMAN':
            for id_ in rl_ids:
                action = agent.compute_action(state[id_])
                actions.update({id_: action})
        state, r, _, _ = env.step(actions)
        outflow.append(env.k.vehicle.get_outflow_rate(600)) # measured by one min

        if i == 0:
            vel_map_list = vel_map_
        else:
            vel_map_list = np.vstack((vel_map_list, vel_map_))

    outflow_lists.append(outflow[-500:])
    vel_lists.append(vel)
    vel_map_list[vel_map_list==0.0] = np.nan
    vel_map_lists.append(vel_map_list)

 Starting SUMO on port 47579


Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 2 seconds


 Starting SUMO on port 47579


Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:47579 [Errno 111] Connection refused
 Retrying in 2 seconds


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
for i in range(6):
    ax.plot(vel_lists[i])
title = AGENT + ' multi-agent mean velocity'
savepath = '../result/MultiAgent/' + AGENT + '/' + (title + ' ' + PATH).replace(' ', '_') + '.png'
title = title + ' \n mean velocity {:3f}'.format(np.mean(vel_lists))
title = title + ' \n mean outflow rate {:3f}'.format(np.mean(outflow_lists))
ax.set_ylabel('system level mean velocity (m/s)')
ax.set_xlabel('step')
fig.suptitle(title)
plt.subplots_adjust(top=0.8)

In [ ]:
fig.savefig(savepath)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 6))
fig.tight_layout()

for i in range(6):
    # plt.subplot(2, 3, i+1)
    x = np.arange(int(env.env_params.horizon))
    y = np.arange(0, int(left_length), step=0.1)
    xx, yy = np.meshgrid(x, y)
    im = axes[i%2, i%3].pcolormesh(xx, yy, vel_map_lists[i].T)
    if i % 3 == 0:
        axes[i%2, i%3].set_ylabel('Position (m)')
    if int(i / 3.0) > 0:
        axes[int(i/3), int(i/3)].set_xlabel('step')
        
clb = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.95)
clb.set_clim(0, 30)
clb.set_label('Velocity (m/s)')
title = AGENT + ' Space-Time Diagram of 600 meter merge road'
fig.suptitle(title)   
plt.subplots_adjust(top=0.9, right=0.8)
savepath = '../result/MultiAgent/' + AGENT + '/' + (title + ' ' + PATH).replace(' ', '_') + '.png'

In [ ]:
fig.savefig(savepath)